In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
input_file_name = "/Users/abuzatu/Work/data/finance/stocks/Revolut/Revolut_trades_2020.txt"
input_file_name_end = "/Users/abuzatu/Work/data/finance/stocks/Revolut/Revolut_trades_2020_end.txt"
input_file_name_deposits = "/Users/abuzatu/Work/data/finance/stocks/Revolut/Revolut_trades_2020_deposits.txt"

In [3]:
def read_file(input_file_name):
    file = open(input_file_name, "r")
    lines = file.readlines()
    for line in lines:
        line = line.rstrip()
        # print(line)
        list_el = line.split()
        if len(list_el) != 5:
            print(list_el)
        stock = float(list_el[2])
        price = float(list_el[3])
        cash = float(list_el[4])
        if ((stock * price) + cash) > 0.01:
            print(list_el)
        if ((stock * price) * cash) > 0:
            print(list_el)
        
# read_file(input_file_name)

In [4]:
def get_date(x):
    list_el = x.split("/")
    return pd.to_datetime(f"2020-{list_el[0]}-{list_el[1]}")

def change_date(df):  
    df["date"] = df.date.map(lambda x: get_date(x))
    df.set_index("date", inplace = True)
    return df

# Deposits

In [5]:
df_deposit = pd.read_csv(input_file_name_deposits, delimiter=" ")
df_deposit

,date,deposit
0,03/23,1073.82
1,03/31,1094.76
2,04/03,393.04
3,04/06,300.00
4,05/13,1100.00
5,05/15,1948.29
6,05/27,770.28
7,06/23,1130.70
8,07/02,508.27
9,07/06,1403.67


In [6]:
df_deposit = change_date(df_deposit)
df_deposit

,deposit
date,
2020-03-23,1073.82
2020-03-31,1094.76
2020-04-03,393.04
2020-04-06,300.00
2020-05-13,1100.00
2020-05-15,1948.29
2020-05-27,770.28
2020-06-23,1130.70
2020-07-02,508.27


In [7]:
df_deposit.deposit.sum()

19456.760000000002

# Trades

In [8]:
df = pd.read_csv(input_file_name, delimiter=" ")
df

,date,ticker,stock,price,cash
0,03/23,TSLA,2.000000,427.3999,-854.80
1,03/31,BA,2.000000,151.5095,-303.02
2,03/31,JNJ,3.000000,131.2300,-393.69
3,04/06,GE,40.000000,7.1495,-287.05
4,04/06,BA,3.000000,136.1095,-409.40
...,...,...,...,...,...
891,12/28,LI,3.000000,26.9490,-80.85
892,12/28,LI,0.710575,26.9500,-19.15
893,12/28,BNTX,1.000000,88.1199,-88.12
894,12/28,BNTX,0.134816,88.1200,-11.88


In [9]:
# check
df["multiply"] = df["stock"] * df["cash"]
df[df["multiply"] > 0.0]

,date,ticker,stock,price,cash,multiply


In [10]:
# the value cash should be used, as it takes into account also the fees paid for the trade
# for the first few months I paid 1 euro per trade, plus the fee from SEC of 0.01$ per trade
df["net"] = (df["stock"] * df["price"]) + df["cash"]
df.sort_values(by="net")

,date,ticker,stock,price,cash,multiply,net
48,06/23,WORK,-100.000000,34.1234,3411.12,-341112.000000,-1.220000
25,06/08,TSLA,-4.000000,948.6601,3793.43,-15173.720000,-1.210400
21,06/05,TSM,-68.000000,55.1900,3751.71,-255116.280000,-1.210000
35,06/11,DAL,-100.000000,28.6655,2865.35,-286535.000000,-1.200000
28,06/09,NCLH,-0.877658,23.5000,19.43,-17.052887,-1.194953
...,...,...,...,...,...,...,...
291,08/11,NIO,220.000000,13.6072,-2993.58,-658587.600000,0.004000
23,06/08,NCLH,92.000000,26.8095,-2466.47,-226915.240000,0.004000
392,08/19,SPCE,58.000000,17.7199,-1027.75,-59609.500000,0.004200
150,07/17,ZM,-11.000000,243.0405,2673.45,-29407.950000,0.004500


In [11]:
# maybe so much used in fees
df.net.sum()

-45.32186518590194

In [12]:
df.drop(["multiply", "net"], axis = 1, inplace = True)

In [13]:
df.head(2)

,date,ticker,stock,price,cash
0,03/23,TSLA,2.0,427.3999,-854.80
1,03/31,BA,2.0,151.5095,-303.02


In [14]:
df = change_date(df)
df

,ticker,stock,price,cash
date,,,,
2020-03-23,TSLA,2.000000,427.3999,-854.80
2020-03-31,BA,2.000000,151.5095,-303.02
2020-03-31,JNJ,3.000000,131.2300,-393.69
2020-04-06,GE,40.000000,7.1495,-287.05
2020-04-06,BA,3.000000,136.1095,-409.40
...,...,...,...,...
2020-12-28,LI,3.000000,26.9490,-80.85
2020-12-28,LI,0.710575,26.9500,-19.15
2020-12-28,BNTX,1.000000,88.1199,-88.12


In [15]:
df.index[0]

Timestamp('2020-03-23 00:00:00')

In [16]:
df["counter"] = 1
df

,ticker,stock,price,cash,counter
date,,,,,
2020-03-23,TSLA,2.000000,427.3999,-854.80,1
2020-03-31,BA,2.000000,151.5095,-303.02,1
2020-03-31,JNJ,3.000000,131.2300,-393.69,1
2020-04-06,GE,40.000000,7.1495,-287.05,1
2020-04-06,BA,3.000000,136.1095,-409.40,1
...,...,...,...,...,...
2020-12-28,LI,3.000000,26.9490,-80.85,1
2020-12-28,LI,0.710575,26.9500,-19.15,1
2020-12-28,BNTX,1.000000,88.1199,-88.12,1


In [17]:
# group by ticker
df1 = df.groupby("ticker").agg("sum")
df1.drop(["price"], axis = 1, inplace = True)
df1["stock"] = df1["stock"].map(lambda x: round(x, 6))
df1

,stock,cash,counter
ticker,,,
AAL,61.012522,-1022.09,22
AAPL,1.256691,-135.53,11
AMD,0.000440,519.44,24
AMZN,0.000000,-9.01,2
AZN,5.956712,-299.98,2
BA,0.007307,-91.69,13
BNTX,20.403537,-2092.56,13
CCL,16.920814,-413.45,23
DAL,0.000000,-438.27,15


In [18]:
# 39 shares traded
len(df1.index.unique())

38

In [19]:
df_AAPL = df[df.ticker == "AAPL"]
df_AAPL

,ticker,stock,price,cash,counter
date,,,,,
2020-07-06,AAPL,5.000000,373.949500,-1869.75,1
2020-07-09,AAPL,-5.000000,380.870000,1904.30,1
2020-07-15,AAPL,4.000000,388.289500,-1553.16,1
2020-07-15,AAPL,0.893250,388.290000,-346.84,1
2020-07-16,AAPL,-2.000000,384.200000,768.40,1
2020-07-16,AAPL,-2.000000,384.200000,768.40,1
2020-07-16,AAPL,-0.893200,384.200000,343.12,1
2020-08-01,AAPL,-0.000050,400.881940,0.02,1
2020-08-01,AAPL,0.000200,100.220485,-0.02,1


In [20]:
df_AAPL.iloc[0:7]

,ticker,stock,price,cash,counter
date,,,,,
2020-07-06,AAPL,5.00000,373.9495,-1869.75,1
2020-07-09,AAPL,-5.00000,380.8700,1904.30,1
2020-07-15,AAPL,4.00000,388.2895,-1553.16,1
2020-07-15,AAPL,0.89325,388.2900,-346.84,1
2020-07-16,AAPL,-2.00000,384.2000,768.40,1
2020-07-16,AAPL,-2.00000,384.2000,768.40,1
2020-07-16,AAPL,-0.89320,384.2000,343.12,1


In [21]:
df_AAPL.iloc[0:7].cash.sum()

14.4699999999998

In [22]:
df_AAPL.iloc[0:7].stock.sum()

4.988999999988586e-05

In [23]:
df_TSLA = df[df.ticker == "TSLA"]
df_TSLA[0:100]

,ticker,stock,price,cash,counter
date,,,,,
2020-03-23,TSLA,2.000000,427.3999,-854.80,1
2020-04-06,TSLA,1.000000,508.9795,-510.05,1
2020-05-01,TSLA,-3.000000,758.8805,2276.58,1
2020-05-12,TSLA,4.000000,810.9995,-3245.08,1
2020-06-08,TSLA,-4.000000,948.6601,3793.43,1
...,...,...,...,...,...
2020-11-23,TSLA,1.000000,522.9599,-522.96,1
2020-11-23,TSLA,0.906857,522.9600,-474.25,1
2020-12-09,TSLA,0.499802,604.9800,-302.37,1


In [24]:
df_TSLA.cash.sum()

99.04999999999586

In [25]:
df_TSLA.stock.sum()

0.006668050000000814

In [26]:
df_9 = df[df.ticker == "MRNA"]
df_9

,ticker,stock,price,cash,counter
date,,,,,
2020-08-18,MRNA,26.000000,66.9895,-1741.73,1
2020-08-18,MRNA,0.645917,66.9900,-43.27,1
2020-08-19,MRNA,-26.000000,68.5100,1781.26,1
2020-08-19,MRNA,-0.645000,68.5100,44.14,1
2020-08-21,MRNA,7.000000,66.8700,-468.09,1
2020-08-21,MRNA,22.000000,66.8700,-1471.14,1
2020-08-21,MRNA,0.908778,66.8700,-60.77,1
2020-08-25,MRNA,-15.000000,65.3100,979.65,1
2020-08-25,MRNA,-14.000000,65.3100,914.34,1


In [27]:
df_9.cash.sum()

398.19999999999953

In [28]:
df1.loc["ENPH"]

stock      0.000818
cash       8.170000
counter    2.000000
Name: ENPH, dtype: float64

In [29]:
df1.cash.sum()

-19456.07000000001

# Stocks at end of period

In [30]:
df_end = pd.read_csv(input_file_name_end, delimiter=" ")
df_end

,ticker,stock_end,unit_cost,total_cost,price_end
0,AAL,61.012522,14.75,900.00,15.77
1,AAPL,1.256691,119.38,150.02,132.69
2,AMD,0.000440,90.83,0.04,91.71
3,AZN,5.956712,50.36,299.98,49.99
4,BA,0.007307,171.07,1.25,214.06
5,BNTX,20.403537,102.56,2092.56,81.52
6,CCL,16.920814,17.73,300.01,21.66
7,ENPH,0.000818,158.95,0.13,175.47
8,F,11.047494,9.06,100.09,8.79
9,FB,0.544847,275.44,150.07,273.16


In [32]:
df_end["value_end"] = df_end["stock_end"] * df_end["price_end"]
df_end

,ticker,stock_end,unit_cost,total_cost,price_end,value_end
0,AAL,61.012522,14.75,900.00,15.77,962.167476
1,AAPL,1.256691,119.38,150.02,132.69,166.750386
2,AMD,0.000440,90.83,0.04,91.71,0.040386
3,AZN,5.956712,50.36,299.98,49.99,297.776016
4,BA,0.007307,171.07,1.25,214.06,1.564164
5,BNTX,20.403537,102.56,2092.56,81.52,1663.296370
6,CCL,16.920814,17.73,300.01,21.66,366.504838
7,ENPH,0.000818,158.95,0.13,175.47,0.143510
8,F,11.047494,9.06,100.09,8.79,97.107473
9,FB,0.544847,275.44,150.07,273.16,148.830437


In [33]:
df_end.value_end.sum()

17926.422895429703

In [34]:
df_end.total_cost.sum()

19360.19

In [35]:
len(df1)

38

In [36]:
len(df_end)

31

In [37]:
df_end

,ticker,stock_end,unit_cost,total_cost,price_end,value_end
0,AAL,61.012522,14.75,900.00,15.77,962.167476
1,AAPL,1.256691,119.38,150.02,132.69,166.750386
2,AMD,0.000440,90.83,0.04,91.71,0.040386
3,AZN,5.956712,50.36,299.98,49.99,297.776016
4,BA,0.007307,171.07,1.25,214.06,1.564164
5,BNTX,20.403537,102.56,2092.56,81.52,1663.296370
6,CCL,16.920814,17.73,300.01,21.66,366.504838
7,ENPH,0.000818,158.95,0.13,175.47,0.143510
8,F,11.047494,9.06,100.09,8.79,97.107473
9,FB,0.544847,275.44,150.07,273.16,148.830437


In [38]:
df2 = df_end.merge(df1, on = "ticker", how = "outer")
df2

,ticker,stock_end,unit_cost,total_cost,price_end,value_end,stock,cash,counter
0,AAL,61.012522,14.75,900.00,15.77,962.167476,61.012522,-1022.09,22
1,AAPL,1.256691,119.38,150.02,132.69,166.750386,1.256691,-135.53,11
2,AMD,0.000440,90.83,0.04,91.71,0.040386,0.000440,519.44,24
3,AZN,5.956712,50.36,299.98,49.99,297.776016,5.956712,-299.98,2
4,BA,0.007307,171.07,1.25,214.06,1.564164,0.007307,-91.69,13
5,BNTX,20.403537,102.56,2092.56,81.52,1663.296370,20.403537,-2092.56,13
6,CCL,16.920814,17.73,300.01,21.66,366.504838,16.920814,-413.45,23
7,ENPH,0.000818,158.95,0.13,175.47,0.143510,0.000818,8.17,2
8,F,11.047494,9.06,100.09,8.79,97.107473,11.047494,-82.35,14
9,FB,0.544847,275.44,150.07,273.16,148.830437,0.544847,-79.61,12


In [39]:
# fill NaN falues with zero
df2.fillna(0.0, inplace = True)
df2

,ticker,stock_end,unit_cost,total_cost,price_end,value_end,stock,cash,counter
0,AAL,61.012522,14.75,900.00,15.77,962.167476,61.012522,-1022.09,22
1,AAPL,1.256691,119.38,150.02,132.69,166.750386,1.256691,-135.53,11
2,AMD,0.000440,90.83,0.04,91.71,0.040386,0.000440,519.44,24
3,AZN,5.956712,50.36,299.98,49.99,297.776016,5.956712,-299.98,2
4,BA,0.007307,171.07,1.25,214.06,1.564164,0.007307,-91.69,13
5,BNTX,20.403537,102.56,2092.56,81.52,1663.296370,20.403537,-2092.56,13
6,CCL,16.920814,17.73,300.01,21.66,366.504838,16.920814,-413.45,23
7,ENPH,0.000818,158.95,0.13,175.47,0.143510,0.000818,8.17,2
8,F,11.047494,9.06,100.09,8.79,97.107473,11.047494,-82.35,14
9,FB,0.544847,275.44,150.07,273.16,148.830437,0.544847,-79.61,12


As expected, the values in the stock column are identifical to those in the stock_end, since we add the trades of 2020, which are from the beginning of trading at Revolut to the end. But let's also check explicitely that it is the same for all

In [44]:
# There should be nothing here
df2[df2["stock_end"] - df2["stock"] > 0.001]

,ticker,stock_end,unit_cost,total_cost,price_end,value_end,stock,cash,counter


In [45]:
df2.head(3)

,ticker,stock_end,unit_cost,total_cost,price_end,value_end,stock,cash,counter
0,AAL,61.012522,14.75,900.00,15.77,962.167476,61.012522,-1022.09,22
1,AAPL,1.256691,119.38,150.02,132.69,166.750386,1.256691,-135.53,11
2,AMD,0.000440,90.83,0.04,91.71,0.040386,0.000440,519.44,24


In [57]:
df2["real_total_cost"] = - df2["cash"]
df2

,ticker,stock_end,unit_cost,total_cost,price_end,value_end,stock,cash,counter,final,final2,unrealised_gain,real_total_cost
17,NCLH,26.416196,22.71,600.04,25.43,671.763872,26.416196,-2775.57,30,-2103.806128,-2175.53,71.723872,2775.57
20,NVAX,0.853609,117.15,100.00,111.51,95.185975,0.853609,-1098.76,74,-1003.574025,-998.76,-4.814025,1098.76
32,DAL,0.000000,0.00,0.00,0.00,0.000000,0.000000,-438.27,15,-438.270000,-438.27,0.000000,438.27
35,RCL,0.000000,0.00,0.00,0.00,0.000000,-0.000000,-401.57,6,-401.570000,-401.57,0.000000,401.57
19,NKLA,0.012669,31.57,0.40,15.26,0.193330,0.012669,-368.52,41,-368.326670,-368.12,-0.206670,368.52
0,AAL,61.012522,14.75,900.00,15.77,962.167476,61.012522,-1022.09,22,-59.922524,-122.09,62.167476,1022.09
6,CCL,16.920814,17.73,300.01,21.66,366.504838,16.920814,-413.45,23,-46.945162,-113.44,66.494838,413.45
4,BA,0.007307,171.07,1.25,214.06,1.564164,0.007307,-91.69,13,-90.125836,-90.44,0.314164,91.69
37,UAL,0.000000,0.00,0.00,0.00,0.000000,0.000000,-87.27,3,-87.270000,-87.27,0.000000,87.27
33,GE,0.000000,0.00,0.00,0.00,0.000000,0.000000,-27.07,2,-27.070000,-27.07,0.000000,27.07


Stocks still had shown at `stock_end` each at average `unit_cost`, with a `total_cost`, and a the `price_end` produce a `value_end`, where `value_end-total_cost` produces the unrealised gain on the stock

The real total gain 

In [59]:
df2["unrealised_gain"] = df2["value_end"] - df2["total_cost"]
df2["total_gain"] = df2["value_end"] - df2["real_total_cost"]
df2["realised_gain"] = df2["total_gain"] - df2["unrealised_gain"]
df2["realised_gain2"] = - df2["real_total_cost"] + df2["total_cost"]
# df2["realised_gain"] = df2["value_end"] + df2["cash"]
df2.head(3)

,ticker,stock_end,unit_cost,total_cost,price_end,value_end,stock,cash,counter,final,final2,unrealised_gain,real_total_cost,total_gain,realised_gain,realised_gain2
17,NCLH,26.416196,22.71,600.04,25.43,671.763872,26.416196,-2775.57,30,-2103.806128,-2175.53,71.723872,2775.57,-2103.806128,-2175.53,-2175.53
20,NVAX,0.853609,117.15,100.00,111.51,95.185975,0.853609,-1098.76,74,-1003.574025,-998.76,-4.814025,1098.76,-1003.574025,-998.76,-998.76
32,DAL,0.000000,0.00,0.00,0.00,0.000000,0.000000,-438.27,15,-438.270000,-438.27,0.000000,438.27,-438.270000,-438.27,-438.27


In [60]:
# for each stock, calculate the value the stock has + the cash I spent on the stock
# to see if overall I am in positive or negative with this stock
df2["final"] = df2["value_end"] + df2["cash"]
# final2 uses the direct cost of buying, not the losses already made in the stock like final does
df2["final2"] = df2["total_cost"] + df2["cash"]
df2["final2"] = df2["final2"].map(lambda x: round(x, 6))
df2

,ticker,stock_end,unit_cost,total_cost,price_end,value_end,stock,cash,counter,final,final2,unrealised_gain,real_total_cost,total_gain,realised_gain,realised_gain2
17,NCLH,26.416196,22.71,600.04,25.43,671.763872,26.416196,-2775.57,30,-2103.806128,-2175.53,71.723872,2775.57,-2103.806128,-2.175530e+03,-2.175530e+03
20,NVAX,0.853609,117.15,100.00,111.51,95.185975,0.853609,-1098.76,74,-1003.574025,-998.76,-4.814025,1098.76,-1003.574025,-9.987600e+02,-9.987600e+02
32,DAL,0.000000,0.00,0.00,0.00,0.000000,0.000000,-438.27,15,-438.270000,-438.27,0.000000,438.27,-438.270000,-4.382700e+02,-4.382700e+02
35,RCL,0.000000,0.00,0.00,0.00,0.000000,-0.000000,-401.57,6,-401.570000,-401.57,0.000000,401.57,-401.570000,-4.015700e+02,-4.015700e+02
19,NKLA,0.012669,31.57,0.40,15.26,0.193330,0.012669,-368.52,41,-368.326670,-368.12,-0.206670,368.52,-368.326670,-3.681200e+02,-3.681200e+02
0,AAL,61.012522,14.75,900.00,15.77,962.167476,61.012522,-1022.09,22,-59.922524,-122.09,62.167476,1022.09,-59.922524,-1.220900e+02,-1.220900e+02
6,CCL,16.920814,17.73,300.01,21.66,366.504838,16.920814,-413.45,23,-46.945162,-113.44,66.494838,413.45,-46.945162,-1.134400e+02,-1.134400e+02
4,BA,0.007307,171.07,1.25,214.06,1.564164,0.007307,-91.69,13,-90.125836,-90.44,0.314164,91.69,-90.125836,-9.044000e+01,-9.044000e+01
37,UAL,0.000000,0.00,0.00,0.00,0.000000,0.000000,-87.27,3,-87.270000,-87.27,0.000000,87.27,-87.270000,-8.727000e+01,-8.727000e+01
33,GE,0.000000,0.00,0.00,0.00,0.000000,0.000000,-27.07,2,-27.070000,-27.07,0.000000,27.07,-27.070000,-2.707000e+01,-2.707000e+01


In [61]:
# sort by final
df2.sort_values(by="final2", inplace = True)
df2

,ticker,stock_end,unit_cost,total_cost,price_end,value_end,stock,cash,counter,final,final2,unrealised_gain,real_total_cost,total_gain,realised_gain,realised_gain2
17,NCLH,26.416196,22.71,600.04,25.43,671.763872,26.416196,-2775.57,30,-2103.806128,-2175.53,71.723872,2775.57,-2103.806128,-2.175530e+03,-2.175530e+03
20,NVAX,0.853609,117.15,100.00,111.51,95.185975,0.853609,-1098.76,74,-1003.574025,-998.76,-4.814025,1098.76,-1003.574025,-9.987600e+02,-9.987600e+02
32,DAL,0.000000,0.00,0.00,0.00,0.000000,0.000000,-438.27,15,-438.270000,-438.27,0.000000,438.27,-438.270000,-4.382700e+02,-4.382700e+02
35,RCL,0.000000,0.00,0.00,0.00,0.000000,-0.000000,-401.57,6,-401.570000,-401.57,0.000000,401.57,-401.570000,-4.015700e+02,-4.015700e+02
19,NKLA,0.012669,31.57,0.40,15.26,0.193330,0.012669,-368.52,41,-368.326670,-368.12,-0.206670,368.52,-368.326670,-3.681200e+02,-3.681200e+02
0,AAL,61.012522,14.75,900.00,15.77,962.167476,61.012522,-1022.09,22,-59.922524,-122.09,62.167476,1022.09,-59.922524,-1.220900e+02,-1.220900e+02
6,CCL,16.920814,17.73,300.01,21.66,366.504838,16.920814,-413.45,23,-46.945162,-113.44,66.494838,413.45,-46.945162,-1.134400e+02,-1.134400e+02
4,BA,0.007307,171.07,1.25,214.06,1.564164,0.007307,-91.69,13,-90.125836,-90.44,0.314164,91.69,-90.125836,-9.044000e+01,-9.044000e+01
37,UAL,0.000000,0.00,0.00,0.00,0.000000,0.000000,-87.27,3,-87.270000,-87.27,0.000000,87.27,-87.270000,-8.727000e+01,-8.727000e+01
33,GE,0.000000,0.00,0.00,0.00,0.000000,0.000000,-27.07,2,-27.070000,-27.07,0.000000,27.07,-27.070000,-2.707000e+01,-2.707000e+01


In [62]:
df2.final.sum()

-1529.6471045703115

In [63]:
df2.final2.sum()

-95.88000000000102

In [64]:
df2.realised_gain.sum()

-95.88000000001216

In [65]:
df2.realised_gain2.sum()

-95.88000000001125

In [67]:
df2.total_gain.sum()

-1529.6471045703115

In [68]:
df2.unrealised_gain.sum()

-1433.7671045702994

In [50]:
df_deposit.deposit.sum()

19456.760000000002

In [51]:
df2.value_end.sum()

17926.4228954297

In [52]:
df2.cash.sum()

-19456.070000000014

In [53]:
df2.total_cost.sum()

19360.190000000002

In [70]:
df2.value_end.sum() - df_deposit.deposit.sum()

-1530.3371045703025

In [71]:
df2.total_cost.sum() - df_deposit.deposit.sum()

-96.56999999999971

# Conclusion

A realised loss of 96.60 euro was obtained at Revolut in calendar year 2020.